# Dataset: Symptoms and COVID Presence (May 2020 data)
Link to a dataset: https://www.kaggle.com/datasets/hemanthhari/symptoms-and-covid-presence

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import make_scorer, accuracy_score, f1_score

from sklearn import model_selection
from sklearn.model_selection import (
    StratifiedKFold,
    cross_validate,
    GridSearchCV,
)

Standard models used:

In [2]:
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

**fcalc** library files are located in **FCALC** folder

**FCALC** folder is located in the same directory as this notebook.

In [3]:
import FCALC.fcalc as fcalc

# Loading data

In [4]:
data_cov = pd.read_csv("Datasets/covid_dataset.csv", 
                       sep=',',
                       true_values=['Yes','yes'],
                       false_values=['No','no']
                      )
data_cov.drop_duplicates(inplace=True)
data_cov.reset_index(inplace=True, drop=True)
data_cov

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,True,True,True,True,True,False,False,False,False,True,...,True,True,False,True,False,True,True,False,False,True
1,True,True,True,True,False,True,True,True,False,False,...,True,False,False,False,True,True,False,False,False,True
2,True,True,True,True,True,True,True,True,False,True,...,True,True,True,False,False,False,False,False,False,True
3,True,True,True,False,False,True,False,False,True,True,...,False,False,True,False,True,True,False,False,False,True
4,True,True,True,True,True,False,True,True,True,True,...,False,True,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,True,False,False,False,False,False,False,True,True,False,...,False,False,False,True,True,True,False,False,False,True
462,True,False,False,False,False,False,False,True,True,False,...,True,False,False,True,True,True,False,False,False,True
463,False,False,True,False,False,True,True,False,True,True,...,False,True,False,True,True,True,False,False,False,True
464,True,False,False,False,True,False,True,True,False,True,...,True,True,False,True,True,True,False,False,False,True


In [5]:
data_cov.shape

(466, 21)

In [6]:
df = data_cov
df

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,True,True,True,True,True,False,False,False,False,True,...,True,True,False,True,False,True,True,False,False,True
1,True,True,True,True,False,True,True,True,False,False,...,True,False,False,False,True,True,False,False,False,True
2,True,True,True,True,True,True,True,True,False,True,...,True,True,True,False,False,False,False,False,False,True
3,True,True,True,False,False,True,False,False,True,True,...,False,False,True,False,True,True,False,False,False,True
4,True,True,True,True,True,False,True,True,True,True,...,False,True,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,True,False,False,False,False,False,False,True,True,False,...,False,False,False,True,True,True,False,False,False,True
462,True,False,False,False,False,False,False,True,True,False,...,True,False,False,True,True,True,False,False,False,True
463,False,False,True,False,False,True,True,False,True,True,...,False,True,False,True,True,True,False,False,False,True
464,True,False,False,False,True,False,True,True,False,True,...,True,True,False,True,True,True,False,False,False,True


In [7]:
df['COVID-19'].value_counts(normalize=True)

COVID-19
True     0.82618
False    0.17382
Name: proportion, dtype: float64

In [8]:
features = df.columns[0:-1]
target = df.columns[-1]

In [9]:
X = df[features]
y = df[target]

# Testing models

In [10]:
scoring = {'accuracy' : make_scorer(accuracy_score),
           'f1_macro' : make_scorer(f1_score, average='macro'),
           'f1_binary' : make_scorer(f1_score),
          }

kfold = StratifiedKFold(n_splits=10, random_state=49, shuffle=True)

In [11]:
def print_results(results):
    acc = np.round(np.mean(results['test_accuracy']), 4) 
    f1_w = np.round(np.mean(results['test_f1_macro']), 4)
    f1_b = np.round(np.mean(results['test_f1_binary']), 4)
    print(f'Accuracy = {acc:0.4f}, F1_macro = {f1_w:0.4f}, F1_binary = {f1_b:0.4f}')  

# Standard models

### Logistic regression

In [12]:
parameters = {'C' : np.linspace(1,5,101)}

model = LogisticRegression()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 101 candidates, totalling 1010 fits


{'C': 1.0}

In [13]:
model = LogisticRegression(C=best.best_params_['C'])
results = cross_validate(estimator=model,
                         X=X, y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9633, F1_macro = 0.9281, F1_binary = 0.9784


### K-Nearest Neighbours

In [14]:
parameters = {'n_neighbors' : range(4,65,4)}

model = KNeighborsClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 16 candidates, totalling 160 fits


{'n_neighbors': 12}

In [15]:
model = KNeighborsClassifier(n_neighbors=best.best_params_['n_neighbors'])
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9376, F1_macro = 0.8679, F1_binary = 0.9637


### Naive Bayes

##### MULTINOMIAL NB

In [16]:
parameters = {'alpha' : np.linspace(0.001,100.001,1001)}

model = MultinomialNB()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 1001 candidates, totalling 10010 fits


{'alpha': 0.501}

In [17]:
model = MultinomialNB(alpha=best.best_params_['alpha'])

results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.8668, F1_macro = 0.6488, F1_binary = 0.9255


##### GAUSSIAN NB

In [18]:
parameters = {'var_smoothing': np.logspace(0,-9, num=100)}

model = GaussianNB()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


{'var_smoothing': 0.1873817422860384}

In [19]:
model = GaussianNB(var_smoothing=best.best_params_['var_smoothing'])
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9634, F1_macro = 0.9313, F1_binary = 0.9782


##### COMPLEMENT NB

In [20]:
parameters = {'alpha' : np.linspace(0.001,100.001,1001)}

model =  ComplementNB()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 1001 candidates, totalling 10010 fits


{'alpha': 60.801}

In [21]:
model = ComplementNB(alpha=best.best_params_['alpha'])
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results) 

Accuracy = 0.9269, F1_macro = 0.8430, F1_binary = 0.9576


### Decision tree

In [22]:
parameters = {'min_samples_split' : range(2,21,2),
              'max_depth' : range(2,17,2),
              'criterion' : ['gini', 'entropy']
             }

model = DecisionTreeClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=1, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 160 candidates, totalling 1600 fits


{'criterion': 'gini', 'max_depth': 6, 'min_samples_split': 2}

In [23]:
model = DecisionTreeClassifier(
    max_depth=best.best_params_['max_depth'], 
    min_samples_split=best.best_params_['min_samples_split'],
    criterion=best.best_params_['criterion']
)
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9506, F1_macro = 0.9045, F1_binary = 0.9708


### RandomForest

In [24]:
parameters = {'n_estimators' : range(40,151,10),
              'min_samples_split' : range(2,11,2),
              'max_depth' : range(4,11,2),
              'criterion' : ['gini', 'entropy']
             }
model = RandomForestClassifier()

best = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    cv=kfold,
                    verbose=3, 
                    scoring=scoring,
                    refit='f1_binary'
                   )
best.fit(X, y)
best.best_params_

Fitting 10 folds for each of 480 candidates, totalling 4800 fits
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.936) f1_binary: (test=0.963) f1_macro: (test=0.866) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.936) f1_binary: (test=0.962) f1_macro: (test=0.881) total time=   0.0s
[CV 4/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.957) f1_binary: (test=0.975) f1_macro: (test=0.916) total time=   0.0s
[CV 5/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accuracy: (test=0.936) f1_binary: (test=0.963) f1_macro: (test=0.866) total time=   0.0s
[CV 6/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=40; accu

[CV 9/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=80; accuracy: (test=0.891) f1_binary: (test=0.938) f1_macro: (test=0.742) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=80; accuracy: (test=0.848) f1_binary: (test=0.916) f1_macro: (test=0.569) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=90; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=90; accuracy: (test=0.936) f1_binary: (test=0.963) f1_macro: (test=0.866) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=90; accuracy: (test=0.915) f1_binary: (test=0.950) f1_macro: (test=0.832) total time=   0.0s
[CV 4/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=90; accuracy: (test=0.957) f1_binary: (test=0.975) f1_macro: (test=0.916

[CV 7/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=130; accuracy: (test=0.913) f1_binary: (test=0.950) f1_macro: (test=0.808) total time=   0.1s
[CV 8/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=130; accuracy: (test=0.935) f1_binary: (test=0.962) f1_macro: (test=0.866) total time=   0.1s
[CV 9/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=130; accuracy: (test=0.891) f1_binary: (test=0.938) f1_macro: (test=0.742) total time=   0.1s
[CV 10/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=130; accuracy: (test=0.848) f1_binary: (test=0.916) f1_macro: (test=0.569) total time=   0.1s
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=140; accuracy: (test=0.957) f1_binary: (test=0.975) f1_macro: (test=0.916) total time=   0.1s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=2, n_estimators=140; accuracy: (test=0.936) f1_binary: (test=0.963) f1_macro: (test

[CV 8/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=60; accuracy: (test=0.935) f1_binary: (test=0.962) f1_macro: (test=0.866) total time=   0.0s
[CV 9/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=60; accuracy: (test=0.870) f1_binary: (test=0.927) f1_macro: (test=0.663) total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=60; accuracy: (test=0.848) f1_binary: (test=0.916) f1_macro: (test=0.569) total time=   0.0s
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=70; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=70; accuracy: (test=0.936) f1_binary: (test=0.963) f1_macro: (test=0.866) total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=70; accuracy: (test=0.915) f1_binary: (test=0.950) f1_macro: (test=0.832

[CV 8/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=110; accuracy: (test=0.935) f1_binary: (test=0.962) f1_macro: (test=0.866) total time=   0.1s
[CV 9/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=110; accuracy: (test=0.891) f1_binary: (test=0.938) f1_macro: (test=0.742) total time=   0.1s
[CV 10/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=110; accuracy: (test=0.870) f1_binary: (test=0.927) f1_macro: (test=0.663) total time=   0.1s
[CV 1/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=120; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.1s
[CV 2/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=120; accuracy: (test=0.936) f1_binary: (test=0.963) f1_macro: (test=0.866) total time=   0.1s
[CV 3/10] END criterion=gini, max_depth=4, min_samples_split=4, n_estimators=120; accuracy: (test=0.957) f1_binary: (test=0.974) f1_macro: (test

...

[CV 7/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=100; accuracy: (test=0.913) f1_binary: (test=0.950) f1_macro: (test=0.808) total time=   0.0s
[CV 8/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=100; accuracy: (test=0.935) f1_binary: (test=0.962) f1_macro: (test=0.866) total time=   0.0s
[CV 9/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=100; accuracy: (test=0.957) f1_binary: (test=0.974) f1_macro: (test=0.916) total time=   0.0s
[CV 10/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=100; accuracy: (test=0.870) f1_binary: (test=0.927) f1_macro: (test=0.663) total time=   0.0s
[CV 1/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=110; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.1s
[CV 2/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=110; accuracy: (test=0.936) f1_binary: (tes

[CV 4/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=150; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.1s
[CV 5/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=150; accuracy: (test=0.957) f1_binary: (test=0.975) f1_macro: (test=0.916) total time=   0.1s
[CV 6/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=150; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.964) total time=   0.1s
[CV 7/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=150; accuracy: (test=0.935) f1_binary: (test=0.962) f1_macro: (test=0.866) total time=   0.1s
[CV 8/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=150; accuracy: (test=0.935) f1_binary: (test=0.962) f1_macro: (test=0.866) total time=   0.1s
[CV 9/10] END criterion=entropy, max_depth=10, min_samples_split=8, n_estimators=150; accuracy: (test=0.957) f1_binary: (test

[CV 1/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=80; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.0s
[CV 2/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=80; accuracy: (test=0.936) f1_binary: (test=0.963) f1_macro: (test=0.866) total time=   0.0s
[CV 3/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=80; accuracy: (test=0.936) f1_binary: (test=0.962) f1_macro: (test=0.881) total time=   0.0s
[CV 4/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=80; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.0s
[CV 5/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=80; accuracy: (test=0.957) f1_binary: (test=0.975) f1_macro: (test=0.916) total time=   0.0s
[CV 6/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=80; accuracy: (test=0.915) f1_binary: (test

[CV 9/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=120; accuracy: (test=0.935) f1_binary: (test=0.962) f1_macro: (test=0.866) total time=   0.1s
[CV 10/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=120; accuracy: (test=0.891) f1_binary: (test=0.938) f1_macro: (test=0.742) total time=   0.1s
[CV 1/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=130; accuracy: (test=0.979) f1_binary: (test=0.987) f1_macro: (test=0.960) total time=   0.1s
[CV 2/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=130; accuracy: (test=0.936) f1_binary: (test=0.963) f1_macro: (test=0.866) total time=   0.1s
[CV 3/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=130; accuracy: (test=0.957) f1_binary: (test=0.974) f1_macro: (test=0.925) total time=   0.1s
[CV 4/10] END criterion=entropy, max_depth=10, min_samples_split=10, n_estimators=130; accuracy: (test=0.979) f1_binary

{'criterion': 'entropy',
 'max_depth': 8,
 'min_samples_split': 6,
 'n_estimators': 40}

In [25]:
model = RandomForestClassifier(
    n_estimators=best.best_params_['n_estimators'],
    min_samples_split=best.best_params_['min_samples_split'],
    max_depth=best.best_params_['max_depth'],
    criterion=best.best_params_['criterion']
)
results = cross_validate(estimator=model,
                         X=X,
                         y=y,
                         cv=kfold,
                         scoring=scoring
                         )
print_results(results)

Accuracy = 0.9420, F1_macro = 0.8829, F1_binary = 0.9659


# Lazy FCA

Due to the multilabel output of **BinarizedBinaryClassifier** (**1** for **True**, **0** for **false** and **-1** for **undefined**) we cannot utilize **f1_score** with **average='binary'**. Therefore, a function that interprets **undefined** as misclassification was implemented.

In [26]:
def compare_with_binary_f1(y_true, y_pred):
    y_tmp = np.concatenate(
        (np.array(y_true)[:,None],np.array(y_pred)[:,None]),
        axis=1
    )
    df_tmp = pd.DataFrame(y_tmp, columns=['y_true','y_pred'])
    df_tmp.y_true = df_tmp.y_true.astype(bool)
    df_tmp['y_new']= ~df_tmp.loc[df_tmp.y_pred==-1]['y_true']
    df_tmp.loc[df_tmp.y_pred!=-1, 'y_new'] = df_tmp.loc[df_tmp.y_pred!=-1,'y_pred'].astype(bool) 
    df_tmp.y_new = df_tmp.y_new.astype(bool)
    return(f1_score(df_tmp.y_true, df_tmp.y_new))

In [27]:
def results_fca(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)
    f1_binary = compare_with_binary_f1(y_true, y_pred)
    return (accuracy, f1, f1_binary)  

## BinarizedBinaryClassifier

**BinarizedBinaryClassifier** is not a **scikit** model, so **GridSearchCV** and **cross_validate** from **scikit** do not support it. Thus, we check desired parameters in **for**  loops.

In [28]:
parameters = {'alpha' : np.linspace(0, 1, 21),
              'method': ['standard','standard-support','ratio-support'],
             }

n = kfold.get_n_splits(X)

f1_best = 0
alpha_best = 0.
method_best = 'standard'

for method in parameters['method']:
    for alpha in parameters['alpha']:
        
        f1 = [0 for i in range(n)]
        for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
            bin_cls = fcalc.classifier.BinarizedBinaryClassifier(
                X.iloc[train_index].values, 
                y.iloc[train_index].to_numpy(),
                method=method,
                alpha=alpha
            )
            bin_cls.predict(X.iloc[test_index].values)
            f1[i] = f1_score(y.iloc[test_index], bin_cls.predictions, average='macro')
            print(f"[CV {i+1}/{n}] method={method}, alpha={alpha:0.2f}, f1_macro={f1[i]:0.4f}") 
                    
        f1_mean = np.mean(f1)
        print(f'f1_mean = {f1_mean:0.4f}\n')
        if(f1_mean > f1_best):
            f1_best = f1_mean
            alpha_best = alpha
            method_best = method
            
print(f"f1_best={f1_best:0.4f}, method={method_best}, alpha={alpha_best:0.2f}")

[CV 1/10] method=standard, alpha=0.00, f1_macro=0.2991
[CV 2/10] method=standard, alpha=0.00, f1_macro=0.3038
[CV 3/10] method=standard, alpha=0.00, f1_macro=0.3122
[CV 4/10] method=standard, alpha=0.00, f1_macro=0.3052
[CV 5/10] method=standard, alpha=0.00, f1_macro=0.3008
[CV 6/10] method=standard, alpha=0.00, f1_macro=0.2991
[CV 7/10] method=standard, alpha=0.00, f1_macro=0.3045
[CV 8/10] method=standard, alpha=0.00, f1_macro=0.2906
[CV 9/10] method=standard, alpha=0.00, f1_macro=0.3083
[CV 10/10] method=standard, alpha=0.00, f1_macro=0.3045
f1_mean = 0.3028

[CV 1/10] method=standard, alpha=0.05, f1_macro=0.3059
[CV 2/10] method=standard, alpha=0.05, f1_macro=0.3836
[CV 3/10] method=standard, alpha=0.05, f1_macro=0.4535
[CV 4/10] method=standard, alpha=0.05, f1_macro=0.3059
[CV 5/10] method=standard, alpha=0.05, f1_macro=0.4535
[CV 6/10] method=standard, alpha=0.05, f1_macro=0.4471
[CV 7/10] method=standard, alpha=0.05, f1_macro=0.4524
[CV 8/10] method=standard, alpha=0.05, f1_macr

[CV 6/10] method=standard, alpha=0.70, f1_macro=0.3263
[CV 7/10] method=standard, alpha=0.70, f1_macro=0.4480
[CV 8/10] method=standard, alpha=0.70, f1_macro=0.2415
[CV 9/10] method=standard, alpha=0.70, f1_macro=0.2465
[CV 10/10] method=standard, alpha=0.70, f1_macro=0.3810
f1_mean = 0.3669

[CV 1/10] method=standard, alpha=0.75, f1_macro=0.4411
[CV 2/10] method=standard, alpha=0.75, f1_macro=0.2898
[CV 3/10] method=standard, alpha=0.75, f1_macro=0.4055
[CV 4/10] method=standard, alpha=0.75, f1_macro=0.4128
[CV 5/10] method=standard, alpha=0.75, f1_macro=0.4618
[CV 6/10] method=standard, alpha=0.75, f1_macro=0.3263
[CV 7/10] method=standard, alpha=0.75, f1_macro=0.4408
[CV 8/10] method=standard, alpha=0.75, f1_macro=0.2353
[CV 9/10] method=standard, alpha=0.75, f1_macro=0.2465
[CV 10/10] method=standard, alpha=0.75, f1_macro=0.3749
f1_mean = 0.3635

[CV 1/10] method=standard, alpha=0.80, f1_macro=0.4171
[CV 2/10] method=standard, alpha=0.80, f1_macro=0.3385
[CV 3/10] method=standard, 

[CV 1/10] method=standard-support, alpha=0.35, f1_macro=0.5554
[CV 2/10] method=standard-support, alpha=0.35, f1_macro=0.4532
[CV 3/10] method=standard-support, alpha=0.35, f1_macro=0.3505
[CV 4/10] method=standard-support, alpha=0.35, f1_macro=0.4125
[CV 5/10] method=standard-support, alpha=0.35, f1_macro=0.4964
[CV 6/10] method=standard-support, alpha=0.35, f1_macro=0.4683
[CV 7/10] method=standard-support, alpha=0.35, f1_macro=0.4363
[CV 8/10] method=standard-support, alpha=0.35, f1_macro=0.3799
[CV 9/10] method=standard-support, alpha=0.35, f1_macro=0.3432
[CV 10/10] method=standard-support, alpha=0.35, f1_macro=0.3684
f1_mean = 0.4264

[CV 1/10] method=standard-support, alpha=0.40, f1_macro=0.4683
[CV 2/10] method=standard-support, alpha=0.40, f1_macro=0.3896
[CV 3/10] method=standard-support, alpha=0.40, f1_macro=0.2893
[CV 4/10] method=standard-support, alpha=0.40, f1_macro=0.2627
[CV 5/10] method=standard-support, alpha=0.40, f1_macro=0.4544
[CV 6/10] method=standard-support, a

[CV 9/10] method=standard-support, alpha=0.95, f1_macro=0.1049
[CV 10/10] method=standard-support, alpha=0.95, f1_macro=0.1518
f1_mean = 0.1618

[CV 1/10] method=standard-support, alpha=1.00, f1_macro=0.1060
[CV 2/10] method=standard-support, alpha=1.00, f1_macro=0.1060
[CV 3/10] method=standard-support, alpha=1.00, f1_macro=0.1132
[CV 4/10] method=standard-support, alpha=1.00, f1_macro=0.1261
[CV 5/10] method=standard-support, alpha=1.00, f1_macro=0.1261
[CV 6/10] method=standard-support, alpha=1.00, f1_macro=0.1642
[CV 7/10] method=standard-support, alpha=1.00, f1_macro=0.0980
[CV 8/10] method=standard-support, alpha=1.00, f1_macro=0.1083
[CV 9/10] method=standard-support, alpha=1.00, f1_macro=0.1154
[CV 10/10] method=standard-support, alpha=1.00, f1_macro=0.1288
f1_mean = 0.1192

[CV 1/10] method=ratio-support, alpha=0.00, f1_macro=0.5699
[CV 2/10] method=ratio-support, alpha=0.00, f1_macro=0.5699
[CV 3/10] method=ratio-support, alpha=0.00, f1_macro=0.4471
[CV 4/10] method=ratio-sup

[CV 1/10] method=ratio-support, alpha=0.60, f1_macro=0.5699
[CV 2/10] method=ratio-support, alpha=0.60, f1_macro=0.5699
[CV 3/10] method=ratio-support, alpha=0.60, f1_macro=0.4535
[CV 4/10] method=ratio-support, alpha=0.60, f1_macro=0.6643
[CV 5/10] method=ratio-support, alpha=0.60, f1_macro=0.4535
[CV 6/10] method=ratio-support, alpha=0.60, f1_macro=0.4471
[CV 7/10] method=ratio-support, alpha=0.60, f1_macro=0.4524
[CV 8/10] method=ratio-support, alpha=0.60, f1_macro=0.4524
[CV 9/10] method=ratio-support, alpha=0.60, f1_macro=0.4524
[CV 10/10] method=ratio-support, alpha=0.60, f1_macro=0.4524
f1_mean = 0.4968

[CV 1/10] method=ratio-support, alpha=0.65, f1_macro=0.5699
[CV 2/10] method=ratio-support, alpha=0.65, f1_macro=0.5699
[CV 3/10] method=ratio-support, alpha=0.65, f1_macro=0.4535
[CV 4/10] method=ratio-support, alpha=0.65, f1_macro=0.6643
[CV 5/10] method=ratio-support, alpha=0.65, f1_macro=0.4535
[CV 6/10] method=ratio-support, alpha=0.65, f1_macro=0.4471
[CV 7/10] method=rati

In [29]:
f1 = [0 for i in range(n)]
accuracy = [0 for i in range(n)]
f1_binary = [0 for i in range(n)]

for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    bin_cls = fcalc.classifier.BinarizedBinaryClassifier(
        X.iloc[train_index].values, 
        y.iloc[train_index].to_numpy(),
        method=method_best,
        alpha=alpha_best
    )
    bin_cls.predict(X.iloc[test_index].values)
    
    accuracy[i], f1[i], f1_binary[i] = results_fca(y.iloc[test_index], bin_cls.predictions)
    
    print(f"[CV {i+1}/{n}] method={method_best}, alpha={alpha_best:0.2f}", 
          f"accuracy={accuracy[i]:0.4f}, f1_macro={f1[i]:0.4f}, f1_binary={f1_binary[i]:0.4f}")

f1_mean = np.mean(f1)
accuracy_mean = np.mean(accuracy)
f1_binary_mean = np.mean(f1_binary)

print(f"\n accuracy={accuracy_mean:0.4f}, f1_macro={f1_mean:0.4f}, f1_binary={f1_binary_mean:0.4f}")

[CV 1/10] method=standard, alpha=0.15 accuracy=0.8723, f1_macro=0.6643, f1_binary=0.9286
[CV 2/10] method=standard, alpha=0.15 accuracy=0.8511, f1_macro=0.5699, f1_binary=0.9176
[CV 3/10] method=standard, alpha=0.15 accuracy=0.8511, f1_macro=0.5699, f1_binary=0.9176
[CV 4/10] method=standard, alpha=0.15 accuracy=0.8511, f1_macro=0.5699, f1_binary=0.9176
[CV 5/10] method=standard, alpha=0.15 accuracy=0.8511, f1_macro=0.5699, f1_binary=0.9176
[CV 6/10] method=standard, alpha=0.15 accuracy=0.8298, f1_macro=0.5524, f1_binary=0.9048
[CV 7/10] method=standard, alpha=0.15 accuracy=0.8478, f1_macro=0.5689, f1_binary=0.9157
[CV 8/10] method=standard, alpha=0.15 accuracy=0.8261, f1_macro=0.4524, f1_binary=0.9048
[CV 9/10] method=standard, alpha=0.15 accuracy=0.8261, f1_macro=0.4524, f1_binary=0.9048
[CV 10/10] method=standard, alpha=0.15 accuracy=0.8261, f1_macro=0.4524, f1_binary=0.9048

 accuracy=0.8432, f1_macro=0.5422, f1_binary=0.9134


## PatternBinaryClassifier

In [30]:
parameters = {'alpha' : np.linspace(0, 1, 21),
              'method': ['standard','standard-support','ratio-support'],
             }

n = kfold.get_n_splits(X)

f1_best_p = 0
alpha_best_p = 0.
method_best_p = 'standard'

for method in parameters['method']:
    for alpha in parameters['alpha']:
        
        f1 = [0 for i in range(n)]
        for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
            bin_cls = fcalc.classifier.PatternBinaryClassifier(
                X.iloc[train_index].values, 
                y.iloc[train_index].to_numpy(),
                method=method,
                alpha=alpha
            )
            bin_cls.predict(X.iloc[test_index].values)
            f1[i] = f1_score(y.iloc[test_index], bin_cls.predictions, average='macro')
            print(f"[CV {i+1}/{n}] method={method}, alpha={alpha:0.2f}, f1_macro={f1[i]:0.4f}") 
                    
        f1_mean = np.mean(f1)
        print(f'f1_mean = {f1_mean:0.4f}\n')
        if(f1_mean > f1_best_p):
            f1_best_p = f1_mean
            alpha_best_p = alpha
            method_best_p = method
            
print(f"f1_best={f1_best_p:0.4f}, method={method_best_p}, alpha={alpha_best_p:0.2f}")

[CV 1/10] method=standard, alpha=0.00, f1_macro=0.9313
[CV 2/10] method=standard, alpha=0.00, f1_macro=0.8626
[CV 3/10] method=standard, alpha=0.00, f1_macro=0.9641
[CV 4/10] method=standard, alpha=0.00, f1_macro=0.9247
[CV 5/10] method=standard, alpha=0.00, f1_macro=0.9161
[CV 6/10] method=standard, alpha=0.00, f1_macro=0.9080
[CV 7/10] method=standard, alpha=0.00, f1_macro=0.8315
[CV 8/10] method=standard, alpha=0.00, f1_macro=0.8315
[CV 9/10] method=standard, alpha=0.00, f1_macro=0.8656
[CV 10/10] method=standard, alpha=0.00, f1_macro=0.7760
f1_mean = 0.8811

[CV 1/10] method=standard, alpha=0.05, f1_macro=0.4071
[CV 2/10] method=standard, alpha=0.05, f1_macro=0.3681
[CV 3/10] method=standard, alpha=0.05, f1_macro=0.4861
[CV 4/10] method=standard, alpha=0.05, f1_macro=0.4703
[CV 5/10] method=standard, alpha=0.05, f1_macro=0.6390
[CV 6/10] method=standard, alpha=0.05, f1_macro=0.4581
[CV 7/10] method=standard, alpha=0.05, f1_macro=0.3677
[CV 8/10] method=standard, alpha=0.05, f1_macr

[CV 6/10] method=standard, alpha=0.70, f1_macro=0.0000
[CV 7/10] method=standard, alpha=0.70, f1_macro=0.0000
[CV 8/10] method=standard, alpha=0.70, f1_macro=0.0000
[CV 9/10] method=standard, alpha=0.70, f1_macro=0.0000
[CV 10/10] method=standard, alpha=0.70, f1_macro=0.0000
f1_mean = 0.0000

[CV 1/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 2/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 3/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 4/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 5/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 6/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 7/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 8/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 9/10] method=standard, alpha=0.75, f1_macro=0.0000
[CV 10/10] method=standard, alpha=0.75, f1_macro=0.0000
f1_mean = 0.0000

[CV 1/10] method=standard, alpha=0.80, f1_macro=0.0000
[CV 2/10] method=standard, alpha=0.80, f1_macro=0.0000
[CV 3/10] method=standard, 

[CV 1/10] method=standard-support, alpha=0.35, f1_macro=0.1455
[CV 2/10] method=standard-support, alpha=0.35, f1_macro=0.1455
[CV 3/10] method=standard-support, alpha=0.35, f1_macro=0.1455
[CV 4/10] method=standard-support, alpha=0.35, f1_macro=0.1455
[CV 5/10] method=standard-support, alpha=0.35, f1_macro=0.1455
[CV 6/10] method=standard-support, alpha=0.35, f1_macro=0.1607
[CV 7/10] method=standard-support, alpha=0.35, f1_macro=0.1481
[CV 8/10] method=standard-support, alpha=0.35, f1_macro=0.1481
[CV 9/10] method=standard-support, alpha=0.35, f1_macro=0.1481
[CV 10/10] method=standard-support, alpha=0.35, f1_macro=0.1481
f1_mean = 0.1481

[CV 1/10] method=standard-support, alpha=0.40, f1_macro=0.1455
[CV 2/10] method=standard-support, alpha=0.40, f1_macro=0.1455
[CV 3/10] method=standard-support, alpha=0.40, f1_macro=0.1455
[CV 4/10] method=standard-support, alpha=0.40, f1_macro=0.1455
[CV 5/10] method=standard-support, alpha=0.40, f1_macro=0.1455
[CV 6/10] method=standard-support, a

[CV 8/10] method=standard-support, alpha=0.95, f1_macro=0.1481
[CV 9/10] method=standard-support, alpha=0.95, f1_macro=0.1481
[CV 10/10] method=standard-support, alpha=0.95, f1_macro=0.1481
f1_mean = 0.1481

[CV 1/10] method=standard-support, alpha=1.00, f1_macro=0.1455
[CV 2/10] method=standard-support, alpha=1.00, f1_macro=0.1455
[CV 3/10] method=standard-support, alpha=1.00, f1_macro=0.1455
[CV 4/10] method=standard-support, alpha=1.00, f1_macro=0.1455
[CV 5/10] method=standard-support, alpha=1.00, f1_macro=0.1455
[CV 6/10] method=standard-support, alpha=1.00, f1_macro=0.1607
[CV 7/10] method=standard-support, alpha=1.00, f1_macro=0.1481
[CV 8/10] method=standard-support, alpha=1.00, f1_macro=0.1481
[CV 9/10] method=standard-support, alpha=1.00, f1_macro=0.1481
[CV 10/10] method=standard-support, alpha=1.00, f1_macro=0.1481
f1_mean = 0.1481

[CV 1/10] method=ratio-support, alpha=0.00, f1_macro=0.5320
[CV 2/10] method=ratio-support, alpha=0.00, f1_macro=0.5853
[CV 3/10] method=ratio-

[CV 10/10] method=ratio-support, alpha=0.55, f1_macro=0.6538
f1_mean = 0.6318

[CV 1/10] method=ratio-support, alpha=0.60, f1_macro=0.5676
[CV 2/10] method=ratio-support, alpha=0.60, f1_macro=0.6083
[CV 3/10] method=ratio-support, alpha=0.60, f1_macro=0.6948
[CV 4/10] method=ratio-support, alpha=0.60, f1_macro=0.6759
[CV 5/10] method=ratio-support, alpha=0.60, f1_macro=0.6573
[CV 6/10] method=ratio-support, alpha=0.60, f1_macro=0.6871
[CV 7/10] method=ratio-support, alpha=0.60, f1_macro=0.5865
[CV 8/10] method=ratio-support, alpha=0.60, f1_macro=0.5881
[CV 9/10] method=ratio-support, alpha=0.60, f1_macro=0.6351
[CV 10/10] method=ratio-support, alpha=0.60, f1_macro=0.6538
f1_mean = 0.6354

[CV 1/10] method=ratio-support, alpha=0.65, f1_macro=0.5853
[CV 2/10] method=ratio-support, alpha=0.65, f1_macro=0.6439
[CV 3/10] method=ratio-support, alpha=0.65, f1_macro=0.7548
[CV 4/10] method=ratio-support, alpha=0.65, f1_macro=0.6622
[CV 5/10] method=ratio-support, alpha=0.65, f1_macro=0.6573
[C

In [31]:
f1 = [0 for i in range(n)]
accuracy = [0 for i in range(n)]
f1_binary = [0 for i in range(n)]

for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
    bin_cls = fcalc.classifier.PatternBinaryClassifier(
        X.iloc[train_index].values, 
        y.iloc[train_index].to_numpy(),
        method=method_best_p,
        alpha=alpha_best_p
    )
    bin_cls.predict(X.iloc[test_index].values)
    
    accuracy[i], f1[i], f1_binary[i] = results_fca(y.iloc[test_index], bin_cls.predictions)
    
    print(f"[CV {i+1}/{n}] method={method_best_p}, alpha={alpha_best_p:0.2f}", 
          f"accuracy={accuracy[i]:0.4f}, f1_macro={f1[i]:0.4f}, f1_binary={f1_binary[i]:0.4f}")

f1_mean = np.mean(f1)
accuracy_mean = np.mean(accuracy)
f1_binary_mean = np.mean(f1_binary)

print(f"\n accuracy={accuracy_mean:0.4f}, f1_macro={f1_mean:0.4f}, f1_binary={f1_binary_mean:0.4f}")

[CV 1/10] method=standard, alpha=0.00 accuracy=0.9574, f1_macro=0.9313, f1_binary=0.9737
[CV 2/10] method=standard, alpha=0.00 accuracy=0.9149, f1_macro=0.8626, f1_binary=0.9474
[CV 3/10] method=standard, alpha=0.00 accuracy=0.9787, f1_macro=0.9641, f1_binary=0.9870
[CV 4/10] method=standard, alpha=0.00 accuracy=0.9574, f1_macro=0.9247, f1_binary=0.9744
[CV 5/10] method=standard, alpha=0.00 accuracy=0.9574, f1_macro=0.9161, f1_binary=0.9750
[CV 6/10] method=standard, alpha=0.00 accuracy=0.9362, f1_macro=0.9080, f1_binary=0.9589
[CV 7/10] method=standard, alpha=0.00 accuracy=0.9130, f1_macro=0.8315, f1_binary=0.9487
[CV 8/10] method=standard, alpha=0.00 accuracy=0.9130, f1_macro=0.8315, f1_binary=0.9487
[CV 9/10] method=standard, alpha=0.00 accuracy=0.9348, f1_macro=0.8656, f1_binary=0.9620
[CV 10/10] method=standard, alpha=0.00 accuracy=0.8913, f1_macro=0.7760, f1_binary=0.9367

 accuracy=0.9354, f1_macro=0.8811, f1_binary=0.9612
